# **문자 언어 모델링**
- RNN으로 더 매력적인 문제를 풀어봅시다.
> RNN은  LSTM, GRU(Gated Recurrent Unit)와 같은 다양한 파생형이 있다.
>
>셰익스피어의 작품의로 이루어진 데이터셋을 이용해 학샙을 수행해보자. 이 모델은 자신에게 제공된 선행 문자 뒤에 어떤 문자가 올지 예측하도록 학습해야 한다.

In [0]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None

        if(id is None):
            self.id = np.random.randint(0,1000000000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    return
                    print(self.id)
                    print(self.creation_op)
                    print(len(self.creators))
                    for c in self.creators:
                        print(c.creation_op)
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def sigmoid(self):
        if(self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if(self.autograd):
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
        
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

class Layer(object):
    
    def __init__(self):
        self.parameters = list()
        
    def get_parameters(self):
        return self.parameters

    
class SGD(object):
    
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha
    
    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
        
    def step(self, zero=True):
        
        for p in self.parameters:
            
            p.data -= p.grad.data * self.alpha
            
            if(zero):
                p.grad.data *= 0


class Linear(Layer):

    def __init__(self, n_inputs, n_outputs, bias=True):
        super().__init__()
        
        self.use_bias = bias
        
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/(n_inputs))
        self.weight = Tensor(W, autograd=True)
        if(self.use_bias):
            self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.parameters.append(self.weight)
        
        if(self.use_bias):        
            self.parameters.append(self.bias)

    def forward(self, input):
        if(self.use_bias):
            return input.mm(self.weight)+self.bias.expand(0,len(input.data))
        return input.mm(self.weight)


class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
    
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params


class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        # this random initialiation style is just a convention from word2vec
        self.weight = Tensor((np.random.rand(vocab_size, dim) - 0.5) / dim, autograd=True)
        
        self.parameters.append(self.weight)
    
    def forward(self, input):
        return self.weight.index_select(input)


class Tanh(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.tanh()


class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return input.sigmoid()
    

class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, input, target):
        return input.cross_entropy(target)

    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.parameters += self.w_ih.get_parameters()
        self.parameters += self.w_hh.get_parameters()
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
    
class LSTMCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)        
        self.xc = Linear(n_inputs, n_hidden)        
        
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)        
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()

        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()        
        self.parameters += self.ho.get_parameters()        
        self.parameters += self.hc.get_parameters()                
        
        self.parameters += self.w_ho.get_parameters()        
    
    def forward(self, input, hidden):
        
        prev_hidden = hidden[0]        
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()        
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()        
        c = (f * prev_cell) + (i * g)

        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
    
    def init_hidden(self, batch_size=1):
        init_hidden = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_cell = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_hidden.data[:,0] += 1
        init_cell.data[:,0] += 1
        return (init_hidden, init_cell)

In [0]:
import sys,random,math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)

f = open('shakespear.txt','r')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))


앞선 단원에서는 데이터 안에 있는 단어를 이용해 어휘를 만들었다면 이번에는 데이터셋 안의 문자(글자 하나)를 이용해서 만들어 진다. 그렇기 때문에 데이터셋 또한 단어가 아닌 문자에 상응하는 인덱스 리스트로 변환된다. NumPy 배열인 indices가 바로 그 리스트다.

>다음은 RNN 기본 코드로 이코드는 차원수를 8로 RNN 은닉 계층의 크기를 512로 해서 임베딩을 초기화하며, 출력 가중치는 0으로 초기화 하며 마지막으로 교차 엔트로피 손실 함수(CrossEntrypyLoss())와 확률적 경사하강법 최적화기(SGD)를 초기화하는 코드이다.

In [0]:
embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weight.data *= 0

criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

# **부분 역전파의 필요성**
- 문자 100,000개는 역전파하기 어렵습니다.

> RNN 코드에서 가장 어려운 부분 중 하나는 바로 데이터를 공급하는 미니 배치 논리이다. 앞에서 만들었던 신경망들은 내장 for 반복문으로 이를 시행했었다.

더 중요한 부분은 역전파 단계! 입력 데이터에 도달할 때까지 경사도를 계속해서 역전파하며 이 과정을 통해서 신경망은 모든 가중치에 대해 수정하고 전체 입력 예쩨에 대해 정확하게 예측하는 방법을 학습할 수 있었다. 위 코드에서는 입력 예제에 대해 순전파를 시행한 후, loss.backward()를 이용해 입력 데이터까지 경사도를 역전파 한다. 이가 가능한 이유는 동시에 많은 입력 데이터 요소를 넣어주지 않기 때문이다.(앞선 예제에서는 5개 정도 사용)

하지만 이번에 사용할 데이터는 숫자가 너무 많아 역전파가 어렵다. 그렇기 때문에 부분 역전파(truncated backpropagation)를 시행하는데 이는 고정된 단계만큼만 역전파하고 멈추는 것이다. 얼마나 역전파를 할지를 나타내는 길이 or 하나의 조정 가능한 매개변수로서 역할을 한다.

# **부분 역전파**
- 기술적으로 부분 역전파는 신경망을 악화시킵니다.

>부분 역전파의 단점
>
>첫번째 : 신경망이 무언가를 기억하기 위해 학습할 수 있는 거리를 줄인다.
>
>- RNN 은닉 계측 안에 우연히 설정해 둔 타음시탬프 보다 이전에 생긴 잔여 정보가 있을 수 있지만 이전에 정보에 대해서는 유지하도록 모델을 만들기 때문에 경사도를 사용할 수 없다. 그렇기 때문에 설정된 타임스탬프보다 많은 입력 신호에 대한 예측은 학습할 수 없다.
>
>- 언어 모델링에 대한 절단 변수는 bptt라고 이름 지었으며 이 변수의 값은 대개 16에서 64 사이의 크기로 설정한다.
>
> 아래 코드 : 단어 갯수를 16개로 설정

In [0]:
batch_size = 16
bptt = 25
n_batches = int((indices.shape[0] / (batch_size)))

> 부분 역전파의 단점
>
> 두번째 : 미니 배치 논리를 복잡하게 만든다.
>
>- 부분 역전파를 사용하기 위해서는 커다란 데이터셋 하나가 아닌, bptt 크기의 작은 데이터셋 여러 개를 가지고 있다고 생각해야 하기 때문에 데이터셋을 묶어주는 작업이 필요하다.

In [0]:
#데이터셋에 bptt 적용하기

#--- 1번째 ---#
#데이터셋을 batch_size와 n_batches 간의 배수로 만들어 줌
#텐서로 그룹화해서 넣을 때 데이터 셋이 정방형이 됨.
trimmed_indices = indices[:n_batches*batch_size]

#--- 2번째 ---#
#데이터셋의 모양의바꿔 각 역이 초기 indices 배열의 일부가 되도록 만들어 주는 과정
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

#input과 target에 bptt를 적용해 reshape한 값
n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)
min_loss = 1000

In [0]:
batch_size = 8

print(raw[0:5])

That,


In [0]:
print(indices[0:5])

[31 36 34  2 32]


In [0]:
print(batched_indices[0:5])

[[31 58  5 14 46  5  8 20 55  1 31 16 20 57 20  2]
 [36 43 46 11 61 46 11 20  2 46 13 46 20 46 46 36]
 [34 31 61  2  5  2 37 46 46 36 53  5 46  2 14 25]
 [ 2  4  4 43 20 36  2 34  4 34 51 47 36 36  5 58]
 [32 18 20 18 20  4  5 20  2 47 58  5  4  5 39 58]]


batch_size를 8로 지정해줬기 때문에 열 개수가 8개이다. 이제 이 텐서는 각 길이가 bptt인 소형 데이터셋의 리스트를 구축하는 데에 사용된다.

In [0]:
#입력과 목표가 어떻게 구성되어있는지 print
input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

In [0]:
#첫번째 줄부터 5번째 줄까지
print(input_batches[0][0:5])

[[31 58  5 14 46  5  8 20 55  1 31 16 20 57 20  2]
 [36 43 46 11 61 46 11 20  2 46 13 46 20 46 46 36]
 [34 31 61  2  5  2 37 46 46 36 53  5 46  2 14 25]
 [ 2  4  4 43 20 36  2 34  4 34 51 47 36 36  5 58]
 [32 18 20 18 20  4  5 20  2 47 58  5  4  5 39 58]]


In [0]:
#두번째부터 그 뒤의 5개의 줄을 출력
print(target_batches[0][0:5])

[[36 43 46 11 61 46 11 20  2 46 13 46 20 46 46 36]
 [34 31 61  2  5  2 37 46 46 36 53  5 46  2 14 25]
 [ 2  4  4 43 20 36  2 34  4 34 51 47 36 36  5 58]
 [32 18 20 18 20  4  5 20  2 47 58  5  4  5 39 58]
 [46 46 20 46 32 23 37 20 46  5 12 37 28 28 46  3]]


- 부분 역전파를 이용해서 반복하는 방법을 살펴봅시다.
> 이전 코드와 차이점은 batch_loss를 생성한다는 것!
>
> 매 bptt 단계 후에, 역전파를 수행하고 가중치를 갱신한다. 그 후에는 앞에서와 동일하게 각 에폭마다 리셋되는 은닉 상태를 이용해서 계속 데이터셋을 읽는다.

In [0]:
import sys,random,math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)

# dataset from http://karpathy.github.io/2015/05/21/rnn-effectiveness/
f = open('shakespear.txt','r')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

embed = Embedding(vocab_size=len(vocab),dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weight.data *= 0

criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
#         output.data *= 25
#         temp_dist = output.softmax()
#         temp_dist /= temp_dist.sum()

#         m = (temp_dist > np.random.rand()).argmax()
        m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s

batch_size = 16
bptt = 25
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)
min_loss = 1000

In [0]:
def train(iterations=400):
    global min_loss
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
    #     batches_to_train = 32
        for batch_i in range(batches_to_train):

            hidden = (Tensor(hidden[0].data, autograd=True), Tensor(hidden[1].data, autograd=True))

            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)

                if(t == 0):
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])

            loss = losses[-1]

            loss.backward()
            optim.step()
            total_loss += loss.data / bptt

            epoch_loss = np.exp(total_loss / (batch_i+1))
            if(epoch_loss < min_loss):
                min_loss = epoch_loss
                print()

            log = "\r Iter:" + str(iter)
            log += " - Alpha:" + str(optim.alpha)[0:5]
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Min Loss:" + str(min_loss)[0:5]
            log += " - Loss:" + str(epoch_loss)
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='T').replace("\n"," ")
            if(batch_i % 1 == 0):
                sys.stdout.write(log)
        optim.alpha *= 0.99
    #     print()

In [0]:
train(10)


 Iter:0 - Alpha:0.05 - Batch 1/249 - Min Loss:62.00 - Loss:62.000000000000064 -           eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
 Iter:0 - Alpha:0.05 - Batch 2/249 - Min Loss:61.99 - Loss:61.99926987309552
 Iter:0 - Alpha:0.05 - Batch 3/249 - Min Loss:61.99 - Loss:61.990033922068406
 Iter:0 - Alpha:0.05 - Batch 4/249 - Min Loss:61.97 - Loss:61.97341829681153
 Iter:0 - Alpha:0.05 - Batch 5/249 - Min Loss:61.94 - Loss:61.94267679844616
 Iter:0 - Alpha:0.05 - Batch 6/249 - Min Loss:61.88 - Loss:61.88351535094878
 Iter:0 - Alpha:0.05 - Batch 7/249 - Min Loss:61.78 - Loss:61.7840899697388
 Iter:0 - Alpha:0.05 - Batch 8/249 - Min Loss:61.54 - Loss:61.54726081129713
 Iter:0 - Alpha:0.05 - Batch 9/249 - Min Loss:61.04 - Loss:61.04676609438749
 Iter:0 - Alpha:0.05 - Batch 10/249 - Min Loss:60.31 - Loss:60.31311394679511
 Iter:0 - Alpha:0.05 - Batch 11/249 - Min Loss:58.84 - Loss:58.8409643541845
 Iter:0 - Alpha:0.05 - Batch 12/249 - Min Loss:56.90 - Loss:56.9098016008886


-------------------------
# **출력의 샘플**

다음 코드는 모델을 이용해서 예측을 수행하는 학습 논리의 일부를 사용합니다. 문자열 안에 예측결과를 저장하고 문자열 버전의 출력을 함수에 반환합니다.

In [0]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    print(word2index[init_char])
    print("=================")
    
    for i in range(n):
        print(i)
        print('++++')
        rnn_input = embed.forward(input) #'\n'의 임베딩을 인덱싱
        print("rnn_input : ",rnn_input)
        print()
        output, hidden = model.forward(input=rnn_input, hidden=hidden) # 위에서 학습한 모델 기반으로 순전파(다음단어 예측)
        print("output : ",output)
        print()
        output.data *= 15
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

#         m = (temp_dist > np.random.rand()).argmax() # sample from predictions
        m = output.data.argmax() # take the max prediction
        c = vocab[m]
        print("voca :",c)
        input = Tensor(np.array([m]))
        s += c
    print(len(s))
    print("=========================================================")
    return s

print(generate_sample(n=500, init_char='\n'))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
   2.18939742e-01 -1.50576176e-01  6.60942695e-02  1.97376470e-01
  -2.56671209e-01  2.84068634e-01  1.72268605e-01 -2.12544508e-01
  -2.56242548e-01 -1.49013085e-01  7.27543498e-02 -6.60831365e-02
  -1.83816350e-01  3.22870480e-02  1.48797599e-01 -1.04742528e-01
   9.79299326e-02 -2.03466948e-03 -6.01656631e-02 -1.68613938e-01
   8.07434630e-02 -1.47999997e-01 -1.91221116e-01 -2.31318677e-03
   9.61540965e-05  9.40666889e-04 -1.18888766e-01 -2.05486086e-03
  -1.70813918e-01 -1.14784688e-01  1.66724020e-01  1.09572011e-01
   1.75612089e-03  2.11329042e-03 -1.53035736e-01 -1.65944072e-03
  -3.58908696e-01  2.99932871e-01 -1.00133925e-02  5.54523569e-03
  -2.62720437e-01  9.15760322e-02 -5.97772676e-02  7.66044329e-02
   1.52705105e-01 -1.13277512e-01  6.67123901e-02 -2.04995672e-01
  -5.14719750e-02 -4.01547392e-02  2.32574023e-01 -4.93162305e-02
  -1.15717118e-01  1.22315368e-01  8.63051334e-02  1.08195611e-01
  -3.37020346e-01  1.15905285e-01 -2.543

## 소멸하는 기울기, 폭발하는 기울기

이 신경망에는 추가적인 비선형성이 은닉 상태 생성 절차에 추가되었습니다. 그러므로 순전파는 다음 3단계를 거칩니다.  
1. 이전 은닉상태와 가중치 행렬과의 행렬 곱  
2. 다음 단어의 임베딩과의 덧셈
3. 비선형성 적용(activation function)
이 비선형성은 신경망의 안정성에 중요한 역할을 합니다. 단어 시퀸스 길이와 관계없이(이론적으로는 시간이 지날수록 계속 커지는)은닉 상태는 비선형성의 값(sigmoid 의 경우 0 또는 1)사이에 머물도록 강제당합니다.  
.  .
하지만 이렇게 좋은 속성을 가지지 못한 역전파는 순전파와는 약간 다른 방식으로 일어납니다.  
.  
역전파는 극단적으로 크거나 극단적으로 작은 값으로 이어지는 경향이 있습니다. 큰 값은 발산(수많은 not-a-number[NaN])을 유발하며, 극도로 작은값은 신경망이 학습하지 못하게 만듭니다

-------------------------
## 6. RNN 역전파의 장난감 예제
### 경사의 소멸/폭발을 직접 관찰할 수 있는 예제를 하나 만들어봅시다.
> #### sigmoid와 relu에 대한 순환 역전파 반복문 코드
* sigmoid, relu 각각에 대해 기울기가 아주 작아지거나 커지는지 잘 살펴보자.
* 역전파 과정에서 이 기울기들은 행렬 곱의 결과로써 커지며, sigmoid 활성화 함수의 결과로써 꼬리 부분에서 아주 평평한 미분계수를 가지며 작아진다. ; Gradient Vanishing Problem

In [0]:
import numpy as np

sigmoid = lambda x:1/(1 + np.exp(-x))
relu = lambda x:(x>0).astype(float)*x

weights = np.array([[1,4],[4,1]])
activation = sigmoid(np.array([1,0.01]))

print("Sigmoid Activations")
activations = list()
for iter in range(10):
    activation = sigmoid(activation.dot(weights))
    activations.append(activation)
    print(activation)
print("\nSigmoid Gradients")
gradient = np.ones_like(activation)
for activation in reversed(activations):
    # 활성화 0 또는 1에 근접할 때 sigmoid의 미분계수는 아주 작은 기울기를 낳습니다.
    gradient = (activation * (1 - activation) * gradient)
    gradient = gradient.dot(weights.transpose())
    print(gradient)
    
print("Relu Activations")
activations = list()
for iter in range(10):
    # 행렬곱은 (sigmoid 같은) 비선형에 의해 쪼그라들지 않는 기울기의 폭발을 유발합니다.
    activation = relu(activation.dot(weights))
    activations.append(activation)
    print(activation)
    print("\nRelu Gradients")
    
gradient = np.ones_like(activation)
for activation in reversed(activation):
    gradient = ((activation > 0) * gradient).dot(weights.transpose())
    print(gradient)

Sigmoid Activations
[0.93940638 0.96852968]
[0.9919462  0.99121735]
[0.99301385 0.99302901]
[0.9930713  0.99307098]
[0.99307285 0.99307285]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]
[0.99307291 0.99307291]

Sigmoid Gradients
[0.03439552 0.03439552]
[0.00118305 0.00118305]
[4.06916726e-05 4.06916726e-05]
[1.39961115e-06 1.39961115e-06]
[4.81403643e-08 4.81403637e-08]
[1.65582672e-09 1.65582765e-09]
[5.69682675e-11 5.69667160e-11]
[1.97259346e-12 1.97517920e-12]
[8.45387597e-14 8.02306381e-14]
[1.45938177e-14 2.16938983e-14]
Relu Activations
[4.8135251  4.72615519]

Relu Gradients
[23.71814585 23.98025559]

Relu Gradients
[119.63916823 118.852839  ]

Relu Gradients
[595.05052421 597.40951192]

Relu Gradients
[2984.68857188 2977.61160877]

Relu Gradients
[14895.13500696 14916.36589628]

Relu Gradients
[74560.59859209 74496.90592414]

Relu Gradients
[372548.22228863 372739.30029248]

Relu Gradients
[1863505.42345854 1862932.18944699]

R

## 7. LSTM(Long Short-Term Memory) 셀
### LSTM은 기울기의 소멸/폭발을 해결할 수 있는 산업 표준 모델입니다.

* #### 게이트를 이용한 복사
    - LSTM은 이전 은닉 상태를 복사하고 필요에 따라 정보를 추가하거나 제거함으로써 다음 은닉 상태를 생성
    - 게이트(gate): LSTM이 정보를 추가하고 제거하는 메커니즘

In [0]:
from IPython.display import Image

In [0]:
Image("C:\lstm.png")

In [0]:
Image("C:\lstm_2.png")

* LSTM은 3개의 GATE를 가지고 있고, 이들은 cell state를 보호하고 제어한다.

In [0]:
Image("C:\forget_input.png")

> #### RNN 셀을 위한 순전파 논리

In [0]:
def forward(self, input, hidden):
    from_prev_hidden = self.w_hh.forward(hidden)
    combined = self.w_ih.forward(input) + from_prev_hidden
    new_hidden = self.activation.forward(combined)
    output = self.w_ho.forward(new_hidden)
    return output, new_hidden

> #### LSTM 셀을 위한 순전파 논리 **(cell을 신경써서 볼 것!!)**
* 각 셀은 앞 셀에 u를 더하고, i와 f를 통해 가중치를 적용
* ### f = forget(망각) gate
* f가 0을 취하면, 새로운 셀은 앞에서 본 것을 지워버린다. 
* ### i = input(입력) gate
* i가 1이면, i는 u의 값을 포함시켜서 새로운 셀을 생성
* o = output(출력) gate, 출력 예측이 셀의 상태를 얼마나 관측할 수 있는지를 제어
* o가 전부 0으로만 이뤄져 있으면, self.
* u = update(갱신) vector

In [0]:
def forward(self, input, hidden):
    prev_hidden, prev_cell = (hidden[0], hidden[1])
    
    f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
    print("f:", f)
    i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
    print("i: ", i)
    o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()
    print("o: ", o)
    u = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()
    print("u: ", u)
    
    cell = (f*prev_cell) + (i*u)
    print("cell: ", cell)
    h = o*cell.tanh()
    print("h: ", h)
    output = self.w_ho.forward(h)
    return output, (h, cell)

-------------------------
## LSTM 게이트에 관한 몇 가지 직관
### LSTM 게이트는 의미상으로 메모리에 대한 읽기/쓰기와 비슷하다.
* LSTM에는 게이트 3개 (f, i, o)와 셀 갱신 벡터 1개 (u)가 있다.  
* 이들은 각각 망각(forget), 입력(input), 출력(output), 갱신(update)이다.  
* c 안에 내장되거나 가공되는 모든 정보에 대해 일일이 c를 갱신하지 않아도 행렬곱이나 비선형성을 적용할 수 잇도록 이 4개의 게이트가 보장한다.=> nonlimearity(c)나 c.dot(weights)를 호출하지 않아도 된다는 것.  

하지만, h를 주의해야 한다. 여전히 기울기의 소멸과 폭발에 취약하기 때문.  
* h벡터는 항상 tanh과 sigmoid로 짓이긴 벡터의 조합을 이용해서 만들어지므로 **기울기 폭발은 더 이상 걱정거리가 아니다.**
* 기울기 소멸만 문제
* 하지만 h는 c에 영향을 받는데 c가 장거리에 걸쳐 정보를 운반할 수 있도록 학습할 수 있기 때문에 문제되지 않음.
* => 모든 장거리 정보는 c를 통해 운반되며, h는 c의 지역화된 해설일 뿐이니, h는 출력 예측을 만들고 다음 시간 단계의 게이트 활성화를 구축할 때 유용하다.

## LSTM 계층
### LSTM 구현에 자동 미분 시스템을 사용할 수 있다.

In [0]:
class LSTMCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output

        self.xf = Linear(n_inputs, n_hidden)
        self.xi = Linear(n_inputs, n_hidden)
        self.xo = Linear(n_inputs, n_hidden)        
        self.xc = Linear(n_inputs, n_hidden)        
        
        self.hf = Linear(n_hidden, n_hidden, bias=False)
        self.hi = Linear(n_hidden, n_hidden, bias=False)
        self.ho = Linear(n_hidden, n_hidden, bias=False)
        self.hc = Linear(n_hidden, n_hidden, bias=False)        
        
        self.w_ho = Linear(n_hidden, n_output, bias=False)
        
        self.parameters += self.xf.get_parameters()
        self.parameters += self.xi.get_parameters()
        self.parameters += self.xo.get_parameters()
        self.parameters += self.xc.get_parameters()

        self.parameters += self.hf.get_parameters()
        self.parameters += self.hi.get_parameters()        
        self.parameters += self.ho.get_parameters()        
        self.parameters += self.hc.get_parameters()                
        
        self.parameters += self.w_ho.get_parameters()        

In [0]:
 def forward(self, input, hidden):
        
        prev_hidden = hidden[0]        
        prev_cell = hidden[1]
        
        f = (self.xf.forward(input) + self.hf.forward(prev_hidden)).sigmoid()
        i = (self.xi.forward(input) + self.hi.forward(prev_hidden)).sigmoid()
        o = (self.xo.forward(input) + self.ho.forward(prev_hidden)).sigmoid()        
        g = (self.xc.forward(input) + self.hc.forward(prev_hidden)).tanh()        
        c = (f * prev_cell) + (i * g)

        h = o * c.tanh()
        
        output = self.w_ho.forward(h)
        return output, (h, c)
        
  def init_hidden(self, batch_size=1):
        init_hidden = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_cell = Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
        init_hidden.data[:,0] += 1
        init_cell.data[:,0] += 1
        return (init_hidden, init_cell)

IndentationError: ignored

## 문자 언어 모델 업그레이드하기
### 바닐라 RNN을 LSTM 셀로 바꿔보자.

In [0]:
import sys,random,math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)

# dataset from http://karpathy.github.io/2015/05/21/rnn-effectiveness/
f = open('shakespear.txt','r')
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

embed = Embedding(vocab_size=len(vocab),dim=512)
model = LSTMCell(n_inputs=512, n_hidden=512, n_output=len(vocab))
model.w_ho.weight.data *= 0

criterion = CrossEntropyLoss()
optim = SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)

def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
#         output.data *= 25
#         temp_dist = output.softmax()
#         temp_dist /= temp_dist.sum()

#         m = (temp_dist > np.random.rand()).argmax()
        m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c

In [0]:
batch_size = 16
bptt = 25
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)
min_loss = 1000

In [0]:
target_batches.shape

(249, 25, 16)

In [0]:
def train(iterations=400):
   
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
    #     batches_to_train = 32
        for batch_i in range(batches_to_train):
            global min_loss
            hidden = (Tensor(hidden[0].data, autograd=True), Tensor(hidden[1].data, autograd=True))

            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)

                if(t == 0):
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])

            loss = losses[-1]

            loss.backward()
            optim.step()
            total_loss += loss.data / bptt

            epoch_loss = np.exp(total_loss / (batch_i+1))
            if(epoch_loss < min_loss):
                min_loss = epoch_loss
                print()

            log = "\r Iter:" + str(iter)
            log += " - Alpha:" + str(optim.alpha)[0:5]
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Min Loss:" + str(min_loss)[0:5]
            log += " - Loss:" + str(epoch_loss)
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='T').replace("\n"," ")
            if(batch_i % 1 == 0):
                sys.stdout.write(log)
        optim.alpha *= 0.99
    #     print()

In [34]:
train(10)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
   0.19061204 -0.05097053 -0.24451893  0.06331341  0.21337082  0.21304774
   0.04510619 -0.15116984  0.07060986 -0.43563842  0.03733633  0.00567522
  -0.2670301  -0.02433662 -0.13635667 -0.05897361 -0.26126398 -0.29456234
  -0.00151681 -0.08055816 -0.03094418 -0.08018776 -0.05076904  0.3024217
   0.06945319  0.08118171  0.1429121  -0.28977749  0.17373775 -0.1399656
   0.42825828  0.1549234   0.32085983 -0.02692826 -0.15551277 -0.0672651
  -0.02815066  0.15718598  0.07991909 -0.31973525 -0.21586283 -0.20203266
  -0.01914276  0.01334244 -0.21221479  0.07722897  0.23898107 -0.13943309
   0.00662846 -0.00603532  0.00710361  0.04176171 -0.10216813  0.09045821
   0.09254099 -0.16830605  0.03982617  0.15981911  0.14317511  0.08222426
   0.04148563 -0.22385437  0.19855375 -0.08640537  0.22954116  0.08707606
  -0.28741887  0.03535727 -0.01691825  0.31444904 -0.0485628  -0.21201063
  -0.0281735  -0.10125467  0.17606739  0.20381064 -0.08991262  0.05873603
   0.

KeyboardInterrupt: ignored

## LSTM 문자 언어 모델 학습하기

오리지날 RNN에서 바뀐 부분은 역전파 부분이다. 은닉 벡터가 1개가 아닌 2개이기 때문이다(12번째 라인). 이 코드에서는 시간이 흐르면 alpha가 천천히 감소하도록 했다.

학습이 굉장히 오래 걸린다...!

In [35]:
iterations = 5

for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
    #     batches_to_train = 32
        for batch_i in range(batches_to_train):

            hidden = (Tensor(hidden[0].data, autograd=True), Tensor(hidden[1].data, autograd=True))

            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)

                if(t == 0):
                    losses.append(batch_loss)
                else:
                    losses.append(batch_loss + losses[-1])

            loss = losses[-1]

            loss.backward()
            optim.step()
            total_loss += loss.data / bptt

            epoch_loss = np.exp(total_loss / (batch_i+1))
            if(epoch_loss < min_loss):
                min_loss = epoch_loss
                print()

            log = "\r Iter:" + str(iter)
            log += " - Alpha:" + str(optim.alpha)[0:5]
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Min Loss:" + str(min_loss)[0:5]
            log += " - Loss:" + str(epoch_loss)
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='T').replace("\n"," ")
            if(batch_i % 1 == 0):
                sys.stdout.write(log)
        optim.alpha *= 0.99
    #     print()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  -1.29261059e-01 -8.44399940e-02 -2.04814365e-01 -6.90047297e-02
  -1.93570791e-01 -8.32543897e-02 -6.59738546e-02  2.21644814e-01
   1.01113302e-01  2.38172541e-02  2.00095380e-01  3.04576471e-01
  -1.02634621e-01 -9.08356577e-03  9.75925032e-02  8.00884701e-02
  -4.19014226e-01  1.94207641e-01 -2.31813533e-02  1.23239366e-01
  -2.43951455e-01  9.79197889e-02  1.05155993e-01 -7.59741373e-02
   1.10453994e-01 -1.67399873e-01  9.43759042e-02  9.55678254e-02
  -1.11869111e-01  7.83834686e-02 -7.02067680e-02 -1.27133082e-01
   9.19948793e-03  1.26889952e-02 -9.15942727e-02  1.99046155e-02
   2.03492328e-01  1.66025716e-01 -6.33668693e-02  2.94167244e-01
  -1.15473205e-01  4.80738426e-02 -1.08436453e-01  2.13114161e-02
  -8.27568804e-02 -8.62042177e-02 -8.38175622e-02  4.72193051e-02
  -1.46451046e-01  8.81704210e-02  5.75973541e-02 -4.23894639e-02
   5.87271528e-03  6.74151701e-02  1.37620768e-01 -1.75992891e-03
  -1.74453850e-01  7.76226509e-02 -1.256

KeyboardInterrupt: ignored

## LSTM 문자 언어 모델 튜닝하기

In [36]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 15
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

#         m = (temp_dist > np.random.rand()).argmax() # sample from predictions
        m = output.data.argmax() # take the max prediction
        c = vocab[m]
        print('c :', c)
        input = Tensor(np.array([m]))
        s += c
        print('s :', s)
        print('-' * 20)
    return s
print(generate_sample(n=500, init_char='\n'))

c : T
s : T
--------------------
c : h
s : Th
--------------------
c : a
s : Tha
--------------------
c : t
s : That
--------------------
c :  
s : That 
--------------------
c : w
s : That w
--------------------
c : a
s : That wa
--------------------
c : s
s : That was
--------------------
c :  
s : That was 
--------------------
c : a
s : That was a
--------------------
c : g
s : That was ag
--------------------
c : n
s : That was agn
--------------------
c : o
s : That was agno
--------------------
c : w
s : That was agnow
--------------------
c :  
s : That was agnow 
--------------------
c : o
s : That was agnow o
--------------------
c : n
s : That was agnow on
--------------------
c :  
s : That was agnow on 
--------------------
c : t
s : That was agnow on t
--------------------
c : h
s : That was agnow on th
--------------------
c : a
s : That was agnow on tha
--------------------
c : t
s : That was agnow on that
--------------------
c :  
s : That was agnow on that 
---------